In [1]:
import torch
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import numpy as np
import time
import os

# set default type double
default_type = torch.float32
torch.set_default_dtype(default_type)
torch.set_default_tensor_type('torch.cuda.FloatTensor')


In [2]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]


ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [3]:
print(source_points)

[[61.84826691 40.32822479  4.5       ]
 [99.10094831 54.69940709  2.        ]
 [99.89962676 24.72759871  2.        ]
 [23.54499552 57.03946784  2.        ]
 [25.09781584 22.62636785  2.        ]]


In [4]:
model = PINN([100,100,100])
standard_dev = .1
tfinal = 4.
model.set_location(source_points,[tfinal,x_max,y_max,z_max],source_values=[0,0,0,1,0],sigma = standard_dev)
# tfinal = 10.

d:\andyh\Documents\Projects\mines\methane_project\methane_pinn_dev\Net.py:12: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [5]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-5)
# from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.995)  # Decay LR by 5% every epoch

In [ ]:
from sampling_methods import spread_sample_source
n= int(3e3)
sn = int(5e3)
best_loss = np.inf
print_freq = 100
max_epochs = int(2e4)
sample_freq = 5
cycle_freq = 50
n_source_samples = int(8e2)


for epoch in range(max_epochs):
    start_time = time.time()
    if epoch % sample_freq == 0:
        source_collocation_points = model.source_points(n_source_samples,standard_dev)
        # ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)
        # collocation_points = torch.empty(0,4)
        # ic_col = torch.empty(0,4)
        # ic_col.requires_grad_()
        # gradual_collocation_points = torch.tensor(spread_sample_source(source_points,[0,1,0,0,0],[30,30,1],50,cycle_freq//sample_freq,(epoch%cycle_freq)//sample_freq,10,10)).float()
        gradual_collocation_points = torch.tensor(spread_sample_source(source_points,[0,1,0,0,0],[30,30,1],50,20,20,tfinal,10)).float()

        collocation_points = torch.cat([collocation_points,source_collocation_points])
        wind_vector = torch.ones(len(collocation_points),2)*2
        gradual_wind_vector = torch.ones(len(gradual_collocation_points),2)*2
        # wind_vector[:,0] *= -1
    optimizer.zero_grad()
    ic = torch.cat([torch.zeros(len(collocation_points),1),collocation_points[:,1:]],dim=1)
    ic.requires_grad_

    loss_3 = torch.sum(model.forward(ic)**2)
    loss_1 ,pde_1 = model.compute_pde_loss(collocation_points,wind_vector) # PDE residual loss
    loss_2, pde_2 = model.compute_pde_loss(gradual_collocation_points,gradual_wind_vector)
    # loss_2,pde_2 = model.loss_function(source_collocation_points, uv_points, source_term = source_values) # source term PDE residual loss 
    # loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_collocation_points]),torch.concat([collocation_points,source_collocation_points]))
    # loss = loss_1+loss_3*1

    loss = loss_1#+loss_2
    # loss = loss_3+loss_2
    # loss = loss_1
    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    for p in model.net.parameters():
        # if p.grad.data is not None:
        if p.grad is not None:
            grad_norm += p.grad.data.norm().item()**2
    grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), epoch_time))



epoch: 0, loss: 2.492e+02, grad_norm: 5.781e+03, pde_res: 2.492e+02, time: 4.045e-01
epoch: 100, loss: 1.032e+02, grad_norm: 3.716e+03, pde_res: 1.032e+02, time: 8.826e-03
epoch: 200, loss: 3.523e+01, grad_norm: 2.090e+03, pde_res: 3.523e+01, time: 1.031e-02
epoch: 300, loss: 1.025e+01, grad_norm: 1.006e+03, pde_res: 1.025e+01, time: 9.158e-03
epoch: 400, loss: 3.472e+00, grad_norm: 4.437e+02, pde_res: 3.472e+00, time: 9.001e-03
epoch: 500, loss: 1.494e+00, grad_norm: 2.185e+02, pde_res: 1.494e+00, time: 8.668e-03
epoch: 600, loss: 8.086e-01, grad_norm: 1.228e+02, pde_res: 8.086e-01, time: 8.550e-03
epoch: 700, loss: 5.510e-01, grad_norm: 7.083e+01, pde_res: 5.510e-01, time: 8.933e-03
epoch: 800, loss: 4.020e-01, grad_norm: 5.825e+01, pde_res: 4.020e-01, time: 7.906e-03
epoch: 900, loss: 3.171e-01, grad_norm: 3.970e+01, pde_res: 3.171e-01, time: 8.724e-03
epoch: 1000, loss: 2.632e-01, grad_norm: 2.908e+01, pde_res: 2.632e-01, time: 9.714e-03
epoch: 1100, loss: 2.088e-01, grad_norm: 2.1

KeyboardInterrupt: 

In [7]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = 2
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 500
x_grid = np.linspace(-x_max, 2.0*x_max, n)
y_grid = np.linspace(-y_max, 2.0*y_max, n)
z_grid = np.linspace(0, 2, n)
x_grid = np.linspace(x_min, x_max, n)
y_grid = np.linspace(y_min, y_max, n)
z_grid = np.linspace(0, 2, n)
X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points).float()

time_steps = np.linspace(0, tfinal, 10)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

'\n# Plot the concentration over time\nfor t in time_steps:\n    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input\n    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape\n    \n    plt.figure()\n    plt.contourf(X, Y, concentration, levels=50, cmap=\'viridis\', vmin=0, vmax=10)  # Plot concentration as a contour plot\n    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), \'ro\', label=\'Source Location\')  # Plot the source location\n    plt.colorbar(label=\'Concentration\')\n    # fix colorbar from 0 to 1 \n    # plt.clim(0, 10.0)  # Set colorbar limits\n    plt.title(f"Gas Concentration at t = {t:.2f}")\n    plt.xlabel(\'x\')\n    plt.ylabel(\'y\')\n    # fix colorbar \n    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image\n'

In [8]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t).float()  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=False).cpu().detach().numpy().reshape(n, n)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[1,0], model.source_locs[1,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.title(f"t = {t:.2f}")

    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF

C:\Users\andyh\AppData\Local\Temp\ipykernel_9980\3240819884.py:23: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list


In [ ]:
model.source_mixture
source_term = torch.tensor(np.exp(model.source_mixture.score_samples(model.source_locs)))
source_term

AttributeError: 'Gaussian_Mixture' object has no attribute 'score_samples'

In [ ]:
t_tensor.device
grid_points.device
torch.cat([t_tensor,grid_points],dim=1).device

device(type='cuda', index=0)

In [ ]:
print(model.source_mixture.weights_)

[1]


In [ ]:
model.net

Net(
  (hidden): ModuleList(
    (0): Linear(in_features=4, out_features=100, bias=True)
    (1-2): 2 x Linear(in_features=100, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=1, bias=True)
  )
  (relu): ReLU()
  (tanh): Tanh()
  (leaky_relu): LeakyReLU(negative_slope=0.1)
)